In [306]:
import pandas as pd
from recipe_scrapers import scrape_me
import parse_ingredient
import altair as alt
import numpy as np
from functools import reduce

In [314]:
urls = ['https://www.allrecipes.com/recipe/279938/egg-salad-with-celery/', 'https://www.allrecipes.com/recipe/158968/spinach-and-feta-turkey-burgers/']

In [315]:
def compile_recipe_info(urls):
    dfs = []
    for url in urls:
        scraper = scrape_me(url)
        ingredient_info = parse_ingredients(scraper.ingredients())

        #Add other info
        ingredient_info["recipe"] = scraper.title()
        ingredient_info["cook_time"] = scraper.total_time()
        ingredient_info["link"] = url

        dfs.append(ingredient_info)

    return pd.concat(dfs)

In [309]:
def parse_ingredients(ingredients):
    data = []
    for ingredient in ingredients:
        print(ingredient)
        info = (ingredient, np.NaN, np.NaN, np.NaN)
        try:
            info = parse_ingredient.parse(ingredient)
            info = info.as_dict()
        except Exception as e:
            pass
            data.append(info)
            continue
        
        #puts original ingredient if low confidence
        if info["confidence"] <= .05:
            data.append((ingredient, np.NaN, np.NaN, np.NaN))
            continue
        
        #puts all information together
        data.append((info["product"], info["quantity"], info["unit"], info["usda_info"]["category"]))
        
    return pd.DataFrame(data, columns=["product", "quantity", "unit", "category"])


In [310]:
def merge_shopping_list(list_):
    quantity = list_.groupby("product").quantity.sum()
    units = list_.groupby("product").unit.unique()
    categories = list_.groupby("product").category.first()

    df_merged = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), [quantity, units, categories])
    return df_merged.sort_values(["category", "product"])

In [316]:
list_ = compile_recipe_info(urls)

6 large eggs
None
2 stalks celery, finely chopped
None
½ cup mayonnaise (such as Hellman's®)
None
¼ cup finely chopped onion
None
¼ teaspoon mustard powder
None
⅛ teaspoon salt
None
2 eggs, beaten
None
2 cloves garlic, minced
None
4 ounces feta cheese
None
1 (10 ounce) box frozen chopped spinach, thawed and squeezed dry
None
2 pounds ground turkey
None


In [317]:
final_shopping_list = merge_shopping_list(list_)

In [505]:
list_

,product,quantity,unit,category,recipe,cook_time,link
0,eggs,6.000,None,Dairy and Egg Products,Egg Salad with Celery,25,https://www.allrecipes.com/recipe/279938/egg-s...
1,celery,2.000,stalk,Vegetables and Vegetable Products,Egg Salad with Celery,25,https://www.allrecipes.com/recipe/279938/egg-s...
2,mayonnaise,0.500,cup,Fats and Oils,Egg Salad with Celery,25,https://www.allrecipes.com/recipe/279938/egg-s...
3,onion,0.250,cup,Vegetables and Vegetable Products,Egg Salad with Celery,25,https://www.allrecipes.com/recipe/279938/egg-s...
4,mustard powder,0.250,teaspoon,Spices and Herbs,Egg Salad with Celery,25,https://www.allrecipes.com/recipe/279938/egg-s...
5,salt,0.125,teaspoon,Spices and Herbs,Egg Salad with Celery,25,https://www.allrecipes.com/recipe/279938/egg-s...
0,eggs,2.000,None,Dairy and Egg Products,Spinach and Feta Turkey Burgers,35,https://www.allrecipes.com/recipe/158968/spina...
1,garlic,2.000,clove,None,Spinach and Feta Turkey Burgers,35,https://www.allrecipes.com/recipe/158968/spina...
2,feta cheese,4.000,ounce,Dairy and Egg Products,Spinach and Feta Turkey Burgers,35,https://www.allrecipes.com/recipe/158968/spina...
3,"1 (10 ounce) box frozen chopped spinach, thawe...",NaN,NaN,NaN,Spinach and Feta Turkey Burgers,35,https://www.allrecipes.com/recipe/158968/spina...


In [318]:
final_shopping_list

,quantity,unit,category
product,,,
eggs,8.000,[None],Dairy and Egg Products
feta cheese,4.000,[ounce],Dairy and Egg Products
mayonnaise,0.500,[cup],Fats and Oils
turkey,2.000,[pound],Poultry Products
mustard powder,0.250,[teaspoon],Spices and Herbs
salt,0.125,[teaspoon],Spices and Herbs
celery,2.000,[stalk],Vegetables and Vegetable Products
onion,0.250,[cup],Vegetables and Vegetable Products
"1 (10 ounce) box frozen chopped spinach, thawed and squeezed dry",0.000,[nan],NaN


In [507]:
test = pd.merge(final_shopping_list, list_, left_on="product", right_on="product")

In [509]:
test

,product,quantity_x,unit_x,category_x,quantity_y,unit_y,category_y,recipe,cook_time,link
0,eggs,8.000,[None],Dairy and Egg Products,6.000,None,Dairy and Egg Products,Egg Salad with Celery,25,https://www.allrecipes.com/recipe/279938/egg-s...
1,eggs,8.000,[None],Dairy and Egg Products,2.000,None,Dairy and Egg Products,Spinach and Feta Turkey Burgers,35,https://www.allrecipes.com/recipe/158968/spina...
2,feta cheese,4.000,[ounce],Dairy and Egg Products,4.000,ounce,Dairy and Egg Products,Spinach and Feta Turkey Burgers,35,https://www.allrecipes.com/recipe/158968/spina...
3,mayonnaise,0.500,[cup],Fats and Oils,0.500,cup,Fats and Oils,Egg Salad with Celery,25,https://www.allrecipes.com/recipe/279938/egg-s...
4,turkey,2.000,[pound],Poultry Products,2.000,pound,Poultry Products,Spinach and Feta Turkey Burgers,35,https://www.allrecipes.com/recipe/158968/spina...
5,mustard powder,0.250,[teaspoon],Spices and Herbs,0.250,teaspoon,Spices and Herbs,Egg Salad with Celery,25,https://www.allrecipes.com/recipe/279938/egg-s...
6,salt,0.125,[teaspoon],Spices and Herbs,0.125,teaspoon,Spices and Herbs,Egg Salad with Celery,25,https://www.allrecipes.com/recipe/279938/egg-s...
7,celery,2.000,[stalk],Vegetables and Vegetable Products,2.000,stalk,Vegetables and Vegetable Products,Egg Salad with Celery,25,https://www.allrecipes.com/recipe/279938/egg-s...
8,onion,0.250,[cup],Vegetables and Vegetable Products,0.250,cup,Vegetables and Vegetable Products,Egg Salad with Celery,25,https://www.allrecipes.com/recipe/279938/egg-s...
9,"1 (10 ounce) box frozen chopped spinach, thawe...",0.000,[nan],NaN,NaN,NaN,NaN,Spinach and Feta Turkey Burgers,35,https://www.allrecipes.com/recipe/158968/spina...


In [520]:
selection = alt.selection_single(fields=["recipe"])

ranked_text = alt.Chart(test).mark_text().encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_window(
    rank='rank(row_number)'
)

recipe = ranked_text.encode(text='recipe:N'
).add_selection(selection)

In [521]:
ranked_ingredients = alt.Chart(test.drop_duplicates(subset=["product"])).mark_text().encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_window(
    rank='rank(row_number)'
)

ingredients = ranked_ingredients.encode(text="product:N"
).transform_filter(selection)

In [524]:
chart = alt.hconcat(ingredients, recipe)

In [525]:
chart

alt.HConcatChart(...)

In [390]:
chart.to_dict()

{'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}},
 'hconcat': [{'data': {'name': 'data-81a5227100eeea691c0166edd7d9a10b'},
   'mark': 'text',
   'encoding': {'text': {'type': 'nominal', 'field': 'product'},
    'y': {'type': 'ordinal', 'axis': None, 'field': 'row_number'}},
   'selection': {'selector075': {'type': 'single'}},
   'transform': [{'window': [{'op': 'row_number',
       'field': '',
       'as': 'row_number'}]},
    {'window': [{'op': 'rank', 'field': 'row_number', 'as': 'rank'}]}]},
  {'data': {'name': 'data-a28616c88c9b83655cf92bdd0bdb32d7'},
   'mark': 'text',
   'encoding': {'text': {'type': 'nominal', 'field': 'recipe'},
    'y': {'type': 'ordinal', 'axis': None, 'field': 'row_number'}},
   'title': 'recipe',
   'transform': [{'window': [{'op': 'row_number',
       'field': '',
       'as': 'row_number'}]},
    {'window': [{'op': 'rank', 'field': 'row_number', 'as': 'rank'}]},
    {'lookup': 'product',
     'from': {'data': {'name': 'data-084f0409